In [ ]:
from agent import KalmanSR
from environment import SimpleMDP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from dynamic_programming import value_iteration
import seaborn as sns
%matplotlib notebook

# Set up the problem 

In [ ]:
env = SimpleMDP(5)

In [ ]:
env.create_graph()

In [ ]:
plt.figure()

positions = {0:(0,0), 1:(1,0), 2:(2,0), 3:(3,0), 4:(4,0)}

env.show_graph(layout=positions)

In [ ]:
optimal_policy, optimal_values = value_iteration(env)

In [ ]:
optimal_policy

In [ ]:
optimal_values

In [ ]:
env.get_transition_matrix(optimal_policy)

In [ ]:
M = env.get_successor_representation(optimal_policy, gamma=.9)
M

In [ ]:
plt.figure()
sns.heatmap(M)
plt.title('True successor representation')

# Learning the successor representation using Kalman TD 

In [ ]:
agent = KalmanSR(environment=env)

In [ ]:
all_results = {}
for trial in range(100):
    results = agent.train_one_episode()
    all_results[trial] = results

In [ ]:
results = pd.DataFrame.from_dict({(trial, step): all_results[trial][step] 
                           for trial in all_results.keys() 
                           for step in all_results[trial].keys()},
                       orient='index')

results.index.names = ['trial', 't']

In [ ]:
results.head()

In [ ]:
np.around(results.iloc[-1]['SR'],decimals=3)

In [ ]:
plt.figure()
sns.heatmap(results.iloc[-1]['SR'])
plt.title('Kalman estimate of successor representation')

In [ ]:
results.iloc[-1]['cov']